# Model 



In [ ]:
import re
import os
import time
import numpy
import pickle
import openquake.mbt as mbt
import openquake.mbt.tools.notebook as nb
from prettytable import PrettyTable
from openquake.hazardlib.const import TRT
from openquake.mbt.oqt_project import OQtProject, OQtModel
from openquake.mbt.guis.utils import processing_out
from openquake.mbt.tools import automator
from openquake.mbt.tools.utils import GetSourceIDs
from openquake.mbt.tools.utils import get_time

In [ ]:
prj_path = "./project/test.oqmbtp"
os.environ["OQMBT_PROJECT"] = prj_path

## Load the project and set the active model

In [ ]:
time_start = time_cell = time.time()
project_pickle_filename = os.environ.get('OQMBT_PROJECT')
oqtkp = OQtProject.load_from_file(project_pickle_filename)

In [ ]:
model_id = 'model01'
oqtkp.active_model_id = model_id

if model_id not in oqtkp.models.keys():
    model = OQtModel(model_id='model01', name='Area source based model')
    oqtkp.add_model(model)
    oqtkp.save()

oqtkp = OQtProject.load_from_file(project_pickle_filename)  
model = oqtkp.models[model_id]

In [ ]:
# create reports folder
reports_folder = os.path.join(oqtkp.directory, 'reports/{:s}'.format(model_id))
print(reports_folder)
if not os.path.exists(reports_folder):
    tmp = os.path.join(oqtkp.directory, 'reports')
    if not os.path.exists(tmp):
        os.mkdir(tmp)
    os.mkdir(reports_folder)
    
# Setting variables - Note that paths are relative to the positon of the .oqtkp with the project data
model.area_shapefile_filename = './../data/model01/shp/completeness_superzones_ESHM13.shp'
model.focal_mechanisms_filename = './../data/catalogues/GCMT.ndk'
model.focal_mechanisms_filename = './../data/catalogues/jan76_dec13.ndk'
model.catalogue_csv_filename = './../data/catalogues/CPTI15_v1.5_fixed.csv'

tk_path = re.sub('openquake/mbt', '', os.path.dirname(mbt.__file__))
oqtkp.save()

## Catalogue pre-processing

In [ ]:
from openquake.hmtk.parsers.catalogue.csv_catalogue_parser import CsvCatalogueParser
catalogue_parser = CsvCatalogueParser(os.path.join(oqtkp.directory, model.catalogue_csv_filename))
catalogue = catalogue_parser.read_file()
print(max(catalogue.data['magnitude']))

In [ ]:
model.catalogue_cutoff_magnitude = 4.0
model.mfd_binwidth = 0.1
model.catalogue_maximum_depth = 200.0
model.catalogue_minimum_depth = 0.0
oqtkp.save()

nb_name = 'catalogue_pre_processing.ipynb'
nb_path = 'openquake/mbt/notebooks/catalogue'
nb_full_path = os.path.join(tk_path, nb_path, nb_name)
out = nb.run(nb_full_path, '')
processing_out(out)
time_cell = get_time(time_start, time_cell)

## Area sources
### Load data from shapefile into model
Note that the shapefile attribute table must contain a ID

In [ ]:
print('Loading data from %s' % (model.area_shapefile_filename))
nb_name = 'load_geometry_from_shapefile.ipynb'
nb_path = 'openquake/mbt/notebooks/sources_area'
nb_full_path = os.path.join(tk_path, nb_path, nb_name)
out = nb.run(nb_full_path, '')
processing_out(out)
time_cell = get_time(time_start, time_cell)

#### Refresh model
Refresh the model and set the default value for the upper seismogenic depth. This parameter will be used in one of the folloing processing steps.

In [ ]:
oqtkp = OQtProject.load_from_file(project_pickle_filename)
model = oqtkp.models[model_id]
get_src_ids = GetSourceIDs(model)
oqtkp.save()
time_cell = get_time(time_start, time_cell)

In [ ]:
print('Number of sources in the model: %d ' % (len(model.sources.keys())))

### Set completeness for all the area sources

In [ ]:
oqtkp = OQtProject.load_from_file(project_pickle_filename)
model = oqtkp.models[model_id]
model.compl_data_folder = './../data/model01/completeness'
oqtkp.save()

nb_name = 'set_completeness_to_area_sources.ipynb'
nb_path = 'openquake/mbt/notebooks/sources_area'
nb_full_path = os.path.join(tk_path, nb_path, nb_name)
out = nb.run(nb_full_path,'')
processing_out(out)
time_cell = get_time(time_start, time_cell)

### Set upper seismogenic depth

In [ ]:
oqtkp = OQtProject.load_from_file(project_pickle_filename)
model = oqtkp.models[model_id]
model.upper_seismogenic_depth = 0.0
model.lower_seismogenic_depth = 25.0
oqtkp.save()

nb_name = 'set_upper_seismogenic_depth.ipynb'
nb_path = 'openquake/mbt/notebooks/sources_area'
nb_full_path = os.path.join(tk_path, nb_path, nb_name)
out = nb.run(nb_full_path, '')
processing_out(out)
time_cell = get_time(time_start, time_cell)

### Compute MFD from seismicity

In [ ]:
get_src_ids = GetSourceIDs(model)
get_src_ids.keep_equal_to('source_type', ['AreaSource'])
nb_name = 'compute_double_truncated_GR_from_seismicity.ipynb'
nb_path = 'openquake/mbt/notebooks/sources_area'
nb_full_path = os.path.join(tk_path, nb_path, nb_name)
out = automator.run(project_pickle_filename, model_id, nb_full_path, get_src_ids.keys,
                    reports_folder=reports_folder)
processing_out(out)
time_cell = get_time(time_start, time_cell)

### Hypocentral depth analysis

keep_keys = []
for key in get_src_ids.keys:
    if key.find('sf_') == -1:
        keep_keys.append(key)
print(keep_keys)

In [ ]:
oqtkp = OQtProject.load_from_file(project_pickle_filename)
model = oqtkp.models[model_id]
model.hypo_depth_bin_edges = [0, 7, 15, 25]
model.hypo_dist_filename = 'hypo_depths.hdf5'
oqtkp.save()

if True:
    get_src_ids = GetSourceIDs(model)
    get_src_ids.keep_equal_to('source_type', ['AreaSource'])
    nb_name = 'compute_hypocentral_depth_distribution.ipynb'
    nb_path = 'openquake/mbt/notebooks/sources_area'
    nb_full_path = os.path.join(tk_path, nb_path, nb_name)
    out = automator.run(project_pickle_filename, model_id, nb_full_path, get_src_ids.keys,
                        reports_folder=reports_folder)
    processing_out(out)
    time_cell = get_time(time_start, time_cell)

#### Load hypocentral depths from .csv files

In [ ]:
get_src_ids = GetSourceIDs(model)
get_src_ids.keep_equal_to('source_type', ['AreaSource'])
nb_name = 'load_hypocentral_depth_distribution_from_csv.ipynb'
nb_path = 'openquake/mbt/notebooks/sources_area'
nb_full_path = os.path.join(tk_path, nb_path, nb_name)
out = automator.run(project_pickle_filename, model_id, nb_full_path, get_src_ids.keys,
                    reports_folder='')
processing_out(out)
time_cell = get_time(time_start, time_cell)

### Focal mechanism analysis

In [ ]:
oqtkp = OQtProject.load_from_file(project_pickle_filename)
model = oqtkp.models[model_id]
model.nodal_plane_dist_filename='{:s}_nodal_plane_dist.hdf5'.format(model_id)
oqtkp.save()

In [ ]:
if True:
    get_src_ids = GetSourceIDs(model)
    get_src_ids.keep_equal_to('source_type', ['AreaSource'])
    nb_name = 'compute_focal_mechanism_distribution.ipynb'
    nb_path = 'openquake/mbt/notebooks/sources_area'
    nb_full_path = os.path.join(tk_path, nb_path, nb_name)
    out = automator.run(project_pickle_filename, model_id, nb_full_path, get_src_ids.keys,
                        reports_folder=reports_folder)
    processing_out(out)
    time_cell = get_time(time_start, time_cell)

#### Load nodal plane distribution from .csv files

In [ ]:
time_start = time.time()
get_src_ids = GetSourceIDs(model)
get_src_ids.keep_equal_to('source_type', ['AreaSource'])
nb_name = 'load_nodal_plane_distribution_from_csv.ipynb'
nb_path = 'openquake/mbt/notebooks/sources_area'
nb_full_path = os.path.join(tk_path, nb_path, nb_name)
out = automator.run(project_pickle_filename, model_id, nb_full_path, get_src_ids.keys,
                    reports_folder='')
processing_out(out)
time_cell = get_time(time_start, time_cell)

### Set the MFD for each area source

In [ ]:
oqtkp = OQtProject.load_from_file(project_pickle_filename)
model = oqtkp.models[model_id]
model.magnitude_max_delta = 0.5
oqtkp.save()

time_start = time.time()
get_src_ids = GetSourceIDs(model)
get_src_ids.keep_equal_to('source_type', ['AreaSource'])
nb_name = 'set_mem_from_seismicity_max_obs_plus_delta.ipynb'
nb_path = 'openquake/mbt/notebooks/sources_area'
nb_full_path = os.path.join(tk_path, nb_path, nb_name)
out = automator.run(project_pickle_filename, model_id, nb_full_path, get_src_ids.keys,
                    reports_folder=reports_folder)
processing_out(out)
time_cell = get_time(time_start, time_cell)

In [ ]:
oqtkp = OQtProject.load_from_file(project_pickle_filename)
model = oqtkp.models[model_id]
model.m_min = 4.5
oqtkp.save()
#
time_start = time.time()
get_src_ids = GetSourceIDs(model)
get_src_ids.keep_equal_to('source_type', ['AreaSource'])
nb_name = 'set_mfd_double_truncated_GR.ipynb'
nb_path = 'openquake/mbt/notebooks/sources_area'
nb_full_path = os.path.join(tk_path, nb_path, nb_name)
out = automator.run(project_pickle_filename, model_id, nb_full_path, get_src_ids.keys,
                    reports_folder='')
processing_out(out)
time_cell = get_time(time_start, time_cell)

### GR parameters

In [ ]:
from openquake.mbt.tools.area import create_gr_table
oqtkp = OQtProject.load_from_file(project_pickle_filename)
model_id = oqtkp.active_model_id
model = oqtkp.models[model_id]
print(create_gr_table(model))
time_cell = get_time(time_start, time_cell)

In [ ]:
oqtkp = OQtProject.load_from_file(project_pickle_filename)
model = oqtkp.models[model_id]
for key in sorted(model.sources):
    if model.sources[key].source_type == 'AreaSource':
        src = model.sources[key]
        src.tectonic_region_type = TRT.ACTIVE_SHALLOW_CRUST
        model.sources[key] = src
oqtkp.save()      

In [ ]:
oqtkp = OQtProject.load_from_file(project_pickle_filename)
model = oqtkp.models[model_id]
model.area_discretization = 10.0
model.smoothing_param = [['gaussian', 50, 20, 0.95], ['gaussian', 20,  5, 0.05]]
# model.catalogue_smoothing = 
oqtkp.save()

time_start = time.time()
get_src_ids = GetSourceIDs(model)
get_src_ids.keep_equal_to('source_type', ['AreaSource'])
nb_name = 'create_sources_no_faults.ipynb'
nb_path = 'openquake/mbt/notebooks/sources_distributed_s'
nb_full_path = os.path.join(tk_path, nb_path, nb_name)
out = automator.run(project_pickle_filename, model_id, nb_full_path, get_src_ids.keys,
                    reports_folder=reports_folder)
processing_out(out)
time_cell = get_time(time_start, time_cell)